## Constants

In [1]:
import sys, os
import pandas as pd
import polars as pl
import numpy as np
import subprocess
import gc
import optuna
from datetime import datetime, timezone
import warnings
import xgboost as xgb
import joblib as jl
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import matthews_corrcoef
from mlflow.models import infer_signature
import mlflow
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold

today = datetime.now(timezone.utc).strftime("%Y_%m_%d")
warnings.filterwarnings("ignore")

from hyper_params import (
    mushroom_tuning_2024_08_06_1722934727_params,
)

SEED = 108
random.seed(SEED)
N_FOLDS = 5
# model
is_tunning = True
try:
    rs = subprocess.check_output("nvidia-smi")
    device = "cuda" if rs is not None else "cpu"
except (
    Exception
):  # this command not being found can raise quite a few different errors depending on the configuration
    print("No Nvidia GPU in system!")
    device = "cpu"

best_params = {
    "device": device,
    "verbosity": 0,
    "objective": "binary:logistic",
}
best_params.update(mushroom_tuning_2024_08_06_1722934727_params)
best_params

{'device': 'cuda',
 'verbosity': 0,
 'objective': 'binary:logistic',
 'tree_method': 'hist',
 'eta': 0.0696294726051571,
 'max_depth': 0,
 'min_child_weight': 1,
 'gamma': 0.044230646284796976,
 'subsample': 0.9405269471473167,
 'colsample_bytree': 0.2999355523666192,
 'lambda': 0.9746051811186938,
 'alpha': 4.210861941737071}

## Prepare data

In [2]:
y_train_pkl = jl.load("../y_train.pkl")
X_train_pkl = jl.load("../X_train.pkl").to_numpy()

print(f"train size: {X_train_pkl.shape}")

train size: (3116945, 110)


## CV

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


clf: xgb.XGBClassifier = xgb.XGBClassifier(
    **best_params,
    n_estimators=4000,
    early_stopping_rounds=50,
    enable_categorical=True,
)

In [4]:
from tqdm import tqdm

gc.collect()
skf = StratifiedKFold(n_splits=N_FOLDS)

y_preds = []
y_trues = []
for train_index, test_index in tqdm(skf.split(X_train_pkl, y_train_pkl)):
    X_train, X_test = X_train_pkl[train_index], X_train_pkl[test_index]
    y_train, y_test = y_train_pkl[train_index], y_train_pkl[test_index]

    clf.fit(X=X_train, y=y_train, eval_set=[(X_test, y_test)])

    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)
    y_trues.append(y_test)

    del X_train, X_test, y_train, y_test, y_pred
    gc.collect()
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
print(f"Validation mcc score: {mcc}")

0it [00:00, ?it/s]

[0]	validation_0-logloss:0.63514
[1]	validation_0-logloss:0.58565
[2]	validation_0-logloss:0.53968
[3]	validation_0-logloss:0.51053
[4]	validation_0-logloss:0.47664
[5]	validation_0-logloss:0.44160
[6]	validation_0-logloss:0.40995
[7]	validation_0-logloss:0.38456
[8]	validation_0-logloss:0.35970
[9]	validation_0-logloss:0.33811
[10]	validation_0-logloss:0.31749
[11]	validation_0-logloss:0.30575
[12]	validation_0-logloss:0.28970
[13]	validation_0-logloss:0.27371
[14]	validation_0-logloss:0.25857
[15]	validation_0-logloss:0.25032
[16]	validation_0-logloss:0.23731
[17]	validation_0-logloss:0.22760
[18]	validation_0-logloss:0.21579
[19]	validation_0-logloss:0.20788
[20]	validation_0-logloss:0.19750
[21]	validation_0-logloss:0.18716
[22]	validation_0-logloss:0.17658
[23]	validation_0-logloss:0.16830
[24]	validation_0-logloss:0.16227
[25]	validation_0-logloss:0.15838
[26]	validation_0-logloss:0.15257
[27]	validation_0-logloss:0.14666
[28]	validation_0-logloss:0.14046
[29]	validation_0-loglos

1it [00:32, 32.57s/it]

[0]	validation_0-logloss:0.63514
[1]	validation_0-logloss:0.58566
[2]	validation_0-logloss:0.53971
[3]	validation_0-logloss:0.51051
[4]	validation_0-logloss:0.47660
[5]	validation_0-logloss:0.44159
[6]	validation_0-logloss:0.40996
[7]	validation_0-logloss:0.38457
[8]	validation_0-logloss:0.35973
[9]	validation_0-logloss:0.33812
[10]	validation_0-logloss:0.31751
[11]	validation_0-logloss:0.30579
[12]	validation_0-logloss:0.28974
[13]	validation_0-logloss:0.27374
[14]	validation_0-logloss:0.25860
[15]	validation_0-logloss:0.25033
[16]	validation_0-logloss:0.23732
[17]	validation_0-logloss:0.22762
[18]	validation_0-logloss:0.21581
[19]	validation_0-logloss:0.20791
[20]	validation_0-logloss:0.19753
[21]	validation_0-logloss:0.18721
[22]	validation_0-logloss:0.17663
[23]	validation_0-logloss:0.16835
[24]	validation_0-logloss:0.16230
[25]	validation_0-logloss:0.15841
[26]	validation_0-logloss:0.15260
[27]	validation_0-logloss:0.14668
[28]	validation_0-logloss:0.14050
[29]	validation_0-loglos

2it [01:05, 33.04s/it]

...

[0]	validation_0-logloss:0.63516
[1]	validation_0-logloss:0.58574
[2]	validation_0-logloss:0.53979
[3]	validation_0-logloss:0.51063
[4]	validation_0-logloss:0.47675
[5]	validation_0-logloss:0.44176
[6]	validation_0-logloss:0.41015
[7]	validation_0-logloss:0.38477
[8]	validation_0-logloss:0.35992
[9]	validation_0-logloss:0.33834
[10]	validation_0-logloss:0.31768
[11]	validation_0-logloss:0.30594
[12]	validation_0-logloss:0.28988
[13]	validation_0-logloss:0.27388
[14]	validation_0-logloss:0.25874
[15]	validation_0-logloss:0.25048
[16]	validation_0-logloss:0.23746
[17]	validation_0-logloss:0.22776
[18]	validation_0-logloss:0.21593
[19]	validation_0-logloss:0.20801
[20]	validation_0-logloss:0.19762
[21]	validation_0-logloss:0.18728
[22]	validation_0-logloss:0.17669
[23]	validation_0-logloss:0.16842
[24]	validation_0-logloss:0.16238
[25]	validation_0-logloss:0.15849
[26]	validation_0-logloss:0.15269
[27]	validation_0-logloss:0.14676
[28]	validation_0-logloss:0.14055
[29]	validation_0-loglos

4it [02:14, 33.73s/it]

[0]	validation_0-logloss:0.63678
[1]	validation_0-logloss:0.58713
[2]	validation_0-logloss:0.54097
[3]	validation_0-logloss:0.51169
[4]	validation_0-logloss:0.47765
[5]	validation_0-logloss:0.44250
[6]	validation_0-logloss:0.41074
[7]	validation_0-logloss:0.38527
[8]	validation_0-logloss:0.36034
[9]	validation_0-logloss:0.33867
[10]	validation_0-logloss:0.31797
[11]	validation_0-logloss:0.30617
[12]	validation_0-logloss:0.29008
[13]	validation_0-logloss:0.27401
[14]	validation_0-logloss:0.25884
[15]	validation_0-logloss:0.25055
[16]	validation_0-logloss:0.23751
[17]	validation_0-logloss:0.22778
[18]	validation_0-logloss:0.21592
[19]	validation_0-logloss:0.20800
[20]	validation_0-logloss:0.19759
[21]	validation_0-logloss:0.18722
[22]	validation_0-logloss:0.17658
[23]	validation_0-logloss:0.16826
[24]	validation_0-logloss:0.16221
[25]	validation_0-logloss:0.15830
[26]	validation_0-logloss:0.15249
[27]	validation_0-logloss:0.14655
[28]	validation_0-logloss:0.14033
[29]	validation_0-loglos

5it [02:51, 34.38s/it]


Validation mcc score: 0.9838152713330468


In [5]:
submit_df = jl.load("../submit_df.pkl")
X_test_pkl = jl.load("../X_test.pkl")
lb = jl.load("../lb.pkl")

In [6]:
y_preds = clf.predict(X_test_pkl)
pred_classes = lb.inverse_transform(y_preds)
submit_df["class"] = pred_classes
submit_df.to_csv("submission.csv", index=False)